# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy.crs as ccrs
import geoviews as gv
import pyproj
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,korkuteli,37.0650,30.1957,286.18,56,76,2.42,TR,1683468486
1,1,olonkinbyen,70.9221,-8.7187,273.75,99,100,6.14,SJ,1683468486
2,2,stanley,54.8680,-1.6985,291.22,79,20,3.09,GB,1683468486
3,3,gadzhiyevo,69.2551,33.3362,275.43,94,98,5.02,RU,1683468487
4,4,grytviken,-54.2811,-36.5092,273.28,86,53,6.60,GS,1683468487


In [12]:

def kelvin_to_fahrenheit(temp):
    return (temp - 273.15) * 9/5 + 32

# Convert the 'Max Temp' column from Kelvin to Fahrenheit
city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(kelvin_to_fahrenheit)
city_data_df 


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,korkuteli,37.0650,30.1957,55.454,56,76,2.42,TR,1683468486
1,1,olonkinbyen,70.9221,-8.7187,33.080,99,100,6.14,SJ,1683468486
2,2,stanley,54.8680,-1.6985,64.526,79,20,3.09,GB,1683468486
3,3,gadzhiyevo,69.2551,33.3362,36.104,94,98,5.02,RU,1683468487
4,4,grytviken,-54.2811,-36.5092,32.234,86,53,6.60,GS,1683468487
...,...,...,...,...,...,...,...,...,...,...
571,571,luderitz,-26.6481,15.1594,68.990,50,0,9.63,NaN,1683468675
572,572,urumqi,43.8010,87.6005,52.754,35,0,2.00,CN,1683468511
573,573,priyutnoye,46.1011,43.5072,69.314,43,91,7.11,RU,1683468676
574,574,erenhot,43.6475,111.9767,59.108,20,100,2.68,CN,1683468676


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:

coordinates ={
    'latitude':city_data_df['Lat'],
    'longitude':city_data_df['Lng'],
    'city':city_data_df['City'],
    'humidity':city_data_df['Humidity']
}

coordinates_df = pd.DataFrame(coordinates)


# Configure the map plot
# YOUR CODE HERE
map_plot = coordinates_df.hvplot.points(
    'longitude',
    'latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'humidity',
    scale = 1.5)
# Display the map
map_plot

#coordinates_df

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (humidity)

In [14]:
#humidity_type = coordinates_df['humidity'].dtypes
#print(humidity_type)
#clean_check_df = coordinates_df.dropna()
#clean_check_df

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
conditions_df = city_data_df[(city_data_df["Cloudiness"]<40) & (city_data_df["Humidity"]<50) &
                                (city_data_df["Max Temp"]>60) & (city_data_df["Max Temp"]<80)]


# Drop any rows with null values
conditions_df = conditions_df.dropna()
conditions_df
# Display sample data
conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,diego de almagro,-26.3667,-70.0500,69.458,13,0,0.80,CL,1683468496
55,55,albany,42.6001,-73.9662,66.578,46,9,0.89,US,1683468230
60,60,bethel,41.3712,-73.4140,69.548,47,0,2.57,US,1683468506
81,81,san rafael,-34.6177,-68.3301,65.228,27,4,5.45,AR,1683468513
97,97,kamyzyak,46.1100,48.0736,69.026,49,0,3.00,RU,1683468518
103,103,dzierzgon,53.9220,19.3471,61.772,45,23,3.89,PL,1683468519
168,168,qashar,53.3906,62.9297,78.584,17,30,7.24,KZ,1683468540
174,174,pankrushikha,53.8319,80.3406,68.756,36,8,1.88,RU,1683468542
206,206,general villegas,-35.0328,-63.0125,64.094,40,30,1.42,AR,1683468553
216,216,liaozhong,41.5061,122.7242,60.098,29,0,2.74,CN,1683468556


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
conditions_copy = conditions_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = conditions_copy.loc[:,['City','Country','Lat','Lng','Humidity']]
hotel_df['Hotel'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel
28,diego de almagro,CL,-26.3667,-70.0500,13,
55,albany,US,42.6001,-73.9662,46,
60,bethel,US,41.3712,-73.4140,47,
81,san rafael,AR,-34.6177,-68.3301,27,
97,kamyzyak,RU,46.1100,48.0736,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'radius':radius,
    'categories':'accommodation.hotel',
    'apiKey':geoapify_key,
    #'key':"43172dd314e24eb0a69e400f78a709a5"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

name_address

Starting hotel search
diego de almagro - nearest hotel: Residencial Hambyll
albany - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
san rafael - nearest hotel: Hotel Regional
kamyzyak - nearest hotel: No hotel found
dzierzgon - nearest hotel: No hotel found
qashar - nearest hotel: No hotel found
pankrushikha - nearest hotel: Уют
general villegas - nearest hotel: Rucalén
liaozhong - nearest hotel: No hotel found
freeport - nearest hotel: The Freeport Inn and Marina
dombarovskiy - nearest hotel: No hotel found
ath thawrah - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
palmer - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
arkalyk - nearest hotel: Гостиница "Арсенал"
fallsburg - nearest hotel: Pines Resort
turpan - nearest hotel: Oriental
buka - nearest hotel: No hotel found
chulym - nearest hotel: No hotel found
ixtlan del rio - nearest hotel: El marques de Minas
mount isa - nearest hotel: Ibis Styles
zabol - 

{'type': 'FeatureCollection', 'features': []}

In [18]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
28,diego de almagro,CL,-26.3667,-70.0500,13,,Residencial Hambyll
55,albany,US,42.6001,-73.9662,46,,No hotel found
60,bethel,US,41.3712,-73.4140,47,,Hampton Inn Danbury
81,san rafael,AR,-34.6177,-68.3301,27,,Hotel Regional
97,kamyzyak,RU,46.1100,48.0736,49,,No hotel found
103,dzierzgon,PL,53.9220,19.3471,45,,No hotel found
168,qashar,KZ,53.3906,62.9297,17,,No hotel found
174,pankrushikha,RU,53.8319,80.3406,36,,Уют
206,general villegas,AR,-35.0328,-63.0125,40,,Rucalén
216,liaozhong,CN,41.5061,122.7242,29,,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)